# Matrix Factorization Model End-to-End Demo

This notebook demonstrates KMR's MatrixFactorizationModel for collaborative filtering, including:

- Data generation using KMR utilities
- Model creation and training
- Recommendation generation and evaluation
- Visualization of recommendations and similarities


In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

from kmr.models import MatrixFactorizationModel
from kmr.metrics import AccuracyAtK, PrecisionAtK, RecallAtK
from kmr.utils import KMRDataGenerator, KMRPlotter

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


✅ All imports successful!
TensorFlow version: 2.18.0
Keras version: 3.8.0


## 1. Generate Collaborative Filtering Data

We'll use KMR's data generator to create synthetic user-item interaction data.


In [2]:
print("📦 Generating collaborative filtering data...")

user_ids, item_ids, ratings, user_features, item_features = KMRDataGenerator.generate_collaborative_filtering_data(
    n_users=1000,
    n_items=500,
    n_interactions=10000,
    random_state=42,
    rating_scale=(1, 5),
    sparsity=0.95
)

n_users = len(np.unique(user_ids))
n_items = len(np.unique(item_ids))

print(f"✅ Generated data:")
print(f"   - Users: {n_users}")
print(f"   - Items: {n_items}")
print(f"   - Interactions: {len(user_ids)}")
print(f"   - Rating range: {ratings.min():.1f} - {ratings.max():.1f}")
print(f"   - Average rating: {ratings.mean():.2f}")

# Convert to binary interaction (for implicit feedback)
interactions = (ratings >= 3.0).astype(np.float32)

# Split into train/test
train_size = int(0.8 * len(user_ids))
train_user_ids = tf.constant(user_ids[:train_size])
train_item_ids = tf.constant(item_ids[:train_size])
train_interactions = tf.constant(interactions[:train_size])

test_user_ids = tf.constant(user_ids[train_size:])
test_item_ids = tf.constant(item_ids[train_size:])
test_interactions = tf.constant(interactions[train_size:])


📦 Generating collaborative filtering data...
✅ Generated data:
   - Users: 1000
   - Items: 500
   - Interactions: 10000
   - Rating range: 1.0 - 5.0
   - Average rating: 2.99


## 2. Build Matrix Factorization Model


In [3]:
# Create model
model = MatrixFactorizationModel(
    num_users=n_users,
    num_items=n_items,
    embedding_dim=64,
    top_k=10,
    l2_reg=0.01
)

# Create recommendation metrics
acc_at_5 = AccuracyAtK(k=5, name="acc@5")
acc_at_10 = AccuracyAtK(k=10, name="acc@10")
prec_at_5 = PrecisionAtK(k=5, name="prec@5")
prec_at_10 = PrecisionAtK(k=10, name="prec@10")
recall_at_5 = RecallAtK(k=5, name="recall@5")
recall_at_10 = RecallAtK(k=10, name="recall@10")

# Compile model with recommendation metrics
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=BinaryCrossentropy(),
    metrics=[acc_at_5, acc_at_10, prec_at_5, prec_at_10, recall_at_5, recall_at_10]
)

print("✅ Model created and compiled!")
print(f"   - Embedding dimension: {model.embedding_dim}")
print(f"   - Top-K: {model.top_k}")
print(f"   - Metrics: Accuracy@5, Accuracy@10, Precision@5, Precision@10, Recall@5, Recall@10")


2025-11-05 16:28:29.069 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized CollaborativeUserItemEmbedding with parameters: {'name': 'collaborative_user_item_embedding', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'num_users': 1000, 'num_items': 500, 'embedding_dim': 64, 'l2_reg': 0.01}
2025-11-05 16:28:29.070 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized NormalizedDotProductSimilarity with parameters: {'name': 'normalized_dot_product_similarity', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}
2025-11-05 16:28:29.070 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized TopKRecommendationSelector with parameters: {'name': 'top_k_recommendation_selector', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name

✅ Model created and compiled!
   - Embedding dimension: 64
   - Top-K: 10
   - Metrics: Accuracy@5, Accuracy@10, Precision@5, Precision@10, Recall@5, Recall@10


/Users/piotrlaczkowski/Library/Caches/pypoetry/virtualenvs/kmr-S1SSCx8j-py3.12/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:642: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


## 3. Train Model


In [4]:
print("🚀 Training Model")
print("=" * 60)
print("Using model.fit() with built-in ranking loss")
print("=" * 60)
print("The model's train_step() method handles ranking loss internally!")
print("Just prepare data and call model.fit() - no custom training loop needed.\n")

# Prepare data for keras.fit() format
# Group by user and create batches with all candidate items and binary labels
unique_users = np.unique(train_user_ids.numpy()[:50])  # Use subset for demo
batch_size = 8

# Create training data: for each user, provide all items and binary labels
train_x_user_ids = []
train_x_item_ids = []
train_y = []

for user_id in unique_users:
    if user_id >= n_users:  # Skip invalid user IDs
        continue
    
    user_items = train_item_ids.numpy()[train_user_ids.numpy() == user_id]
    positive_set = set(user_items[user_items < n_items])  # Filter valid items
    
    # Create label vector: 1 for positive items, 0 for others
    labels = np.zeros(n_items, dtype=np.float32)
    labels[list(positive_set)] = 1.0
    
    train_x_user_ids.append(user_id)
    train_x_item_ids.append(np.arange(n_items))
    train_y.append(labels)

train_x_user_ids = np.array(train_x_user_ids, dtype=np.int32)
train_x_item_ids = np.array(train_x_item_ids, dtype=np.int32)
train_y = np.array(train_y, dtype=np.float32)

print(f"Prepared training data: {len(train_x_user_ids)} users")
print(f"  - User IDs shape: {train_x_user_ids.shape}")
print(f"  - Item IDs shape: {train_x_item_ids.shape}")
print(f"  - Labels shape: {train_y.shape}")
print(f"  - Positive items per user: {train_y.sum(axis=1).mean():.1f} on average\n")

# Train using keras.fit() - the model handles ranking loss internally!
print("Training with model.fit()...")
history = model.fit(
    x=[train_x_user_ids, train_x_item_ids],
    y=train_y,
    epochs=3,
    batch_size=batch_size,
    verbose=1
)

print("\n✅ Training completed!")
print(f"Final loss: {history.history['loss'][-1]:.4f}")

# Display recommendation metrics
if 'acc@5' in history.history:
    print("\n📊 Recommendation Metrics:")
    print(f"   - Accuracy@5:  {history.history['acc@5'][-1]:.4f}")
    print(f"   - Accuracy@10: {history.history['acc@10'][-1]:.4f}")
    print(f"   - Precision@5:  {history.history['prec@5'][-1]:.4f}")
    print(f"   - Precision@10: {history.history['prec@10'][-1]:.4f}")
    print(f"   - Recall@5:  {history.history['recall@5'][-1]:.4f}")
    print(f"   - Recall@10: {history.history['recall@10'][-1]:.4f}")

print("\nNote: The model uses margin ranking loss internally.")
print("      Positive items are encouraged to rank higher than negative items.")
print("      Metrics track recommendation quality: Accuracy@K, Precision@K, Recall@K.")


🚀 Training Model
Using model.fit() with built-in ranking loss
The model's train_step() method handles ranking loss internally!
Just prepare data and call model.fit() - no custom training loop needed.

Prepared training data: 48 users
  - User IDs shape: (48,)
  - Item IDs shape: (48, 500)
  - Labels shape: (48, 500)
  - Positive items per user: 8.9 on average

Training with model.fit()...
Epoch 1/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - acc@10: 0.0723 - acc@5: 0.0723 - loss: 1.5447 - prec@10: 0.0072 - prec@5: 0.0072 - recall@10: 0.0066 - recall@5: 0.0066                          
Epoch 2/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 0.0598 - acc@5: 0.0598 - loss: 1.5447 - prec@10: 0.0060 - prec@5: 0.0060 - recall@10: 0.0050 - recall@5: 0.0050                        
Epoch 3/3
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 0.0878 - acc@5: 0.0878 - loss: 1.5447 - prec@10: 0.0088 - prec@5: 0.0088 - recall@10: 0.0076 - recall@5: 0.0076                        

✅ Training completed!
Fina

## 4. Generate Recommendations and Visualize


In [5]:
# Generate recommendations for a sample user
sample_user_id = tf.constant([0])
sample_item_ids = tf.constant(np.arange(n_items).reshape(1, -1))

rec_indices, rec_scores = model([sample_user_id, sample_item_ids], training=False)

print(f"✅ Generated recommendations for user {sample_user_id[0].numpy()}")
print(f"   Top-{model.top_k} recommended items: {rec_indices[0].numpy()}")
print(f"   Recommendation scores: {rec_scores[0].numpy()}")

# Visualize recommendation scores
fig = KMRPlotter.plot_recommendation_scores(
    rec_scores[0].numpy(),
    top_k=model.top_k,
    title="Recommendation Scores for Sample User"
)
fig.show()


✅ Generated recommendations for user 0
   Top-10 recommended items: [ 69  29 229 151 382  85 247 379 480 383]
   Recommendation scores: [0.33571166 0.33191717 0.32217562 0.31300083 0.2931265  0.29256985
 0.28900632 0.26912594 0.26834485 0.26404378]


/Users/piotrlaczkowski/Library/Caches/pypoetry/virtualenvs/kmr-S1SSCx8j-py3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'matrix_factorization_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
